# Основы программирования в Python

*Алла Тамбовцева, НИУ ВШЭ*

### Работа с API: пример с API ВКонтакте

Сегодня мы немного поработаем с API. API ‒ программный интерфейс приложения, сокращение от *Application Programming Interface*. Этот интерфейс позволяет выполнять различные операции автоматически, через приложение. Если API нам нужен исключительно как источник данных, можно писать запросы, позволяющие обратиться к хранилищу информации внутри API. Если мы хотим управлять приложением, которое будет выполнять какие-то действия, удаленно, можно написать код, который будет, например, автоматически отвечать на сообщения, когда мы не онлайн, лайкать новый пост друга через 30 секунд после его появления, пересылать на почту фотографии, которые выложили участники диалога и прочее.

Мы будем работать с API социальной сети ВКонтакте. Использовать API для написания и приема сообщений средствами Python мы не будем, а рассмотрим API как источник данных, позволяющий выгрузить все посты со стены пользователя или сообщества. Для работы нам понадобится библиотека `vk`, ее нужно установить через `pip install vk` в *Anaconda Command Prompt* (см. пример с `selenium` [здесь](https://nbviewer.jupyter.org/github/allatambov/Py-programming-3/blob/master/11-06/lect-selenium1.ipynb)).

Импортируем библиотеку:

In [1]:
import vk

Теперь нужно авторизоваться: создать приложение и получить токен доступа к нему.

#### Для авторизации:
    
1. [Создать](https://vk.com/apps?act=manage) приложение ВКонтакте (пройти по ссылке). Дать название, выбрать *Standalone-приложение*.
2. Включить приложение, сделать публичным (*Настройки - Состояние* и выбрать в выпадающем меню *Приложение включено и видно всем*).
3. Авторизоваться: скопировать строку ниже в браузер, в `client id` вместо 1 выставить `id` своего приложения (первая строка в настроках ‒ *ID приложения*). Если не хочется ни в чем ограничивать свое приложение, можно оставить `scope=all` (у приложения будет доступ ко всему, к чему есть доступ у пользователя).

    > ```https://oauth.vk.com/authorize?client_id=1&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token```

4. Скопировать `access token` из обновленной адресной строки (все после `access_token=` и до `&expires_in`, без `&`). Никому не показывать! По этому токену можно получить доступ ко всему аккаунту.

In [2]:
# скопировать свой access token без пробелов вместо 1234
token = '1234'

In [3]:
session = vk.Session(access_token = token) # открыть сессию для работы
api = vk.API(session) # подключиться к API

Попробуем сгрузить первые 100 постов со [стены](https://vk.com/hseofficial) неофициального сообщества ВШЭ. Сохраним ссылку на сообщество в переменную `group`:

In [4]:
group = 'hseofficial'

Обратите внимание: ссылка должна быть относительной, то есть без части с `https:/vk.com/`. Python и так будет знать, что мы работаем с сетью ВКонтакте.

Теперь получим доступ к стене этого сообщества:

In [5]:
# wall.get - "подключаемся" к стене
# count - сколько постов выгрузить (максимум)
# v - версия API, можно обойтись без нее, но Python может выдать warning

res = api.wall.get(domain = group, count = 100, v = 5.73)

In [ ]:
res

Результат, сохраненный в `res`, очень похож на словарь. На самом деле, многие API возвращают результаты в формате JSON (JavaScript Object Notation), который тоже может быть представлен в виде набора пар ключ-значение.

In [9]:
res.keys()

dict_keys(['count', 'items'])

Ключами являются `count` и `items`. Нужные нам объекты (текст постов, id автора, дата и время публикации и проч.) находятся в `items`.

In [14]:
res['items'][0] # первый элемент items - первый пост со всей информацией о нем

{'comments': {'can_post': 0, 'count': 0, 'groups_can_post': True},
 'copy_history': [{'attachments': [{'photo': {'access_key': 'dd5969ed596f9c609c',
      'album_id': -7,
      'date': 1527671074,
      'height': 1067,
      'id': 456239082,
      'owner_id': -40176496,
      'photo_1280': 'https://pp.userapi.com/c834402/v834402946/14ed99/Yx1LEJ8gOIo.jpg',
      'photo_130': 'https://pp.userapi.com/c834402/v834402946/14ed96/F2wCFaAhPkI.jpg',
      'photo_2560': 'https://pp.userapi.com/c834402/v834402946/14ed9a/-aMToQjO9XI.jpg',
      'photo_604': 'https://pp.userapi.com/c834402/v834402946/14ed97/OfYh831fPiU.jpg',
      'photo_75': 'https://pp.userapi.com/c834402/v834402946/14ed95/ilhUXWnEe18.jpg',
      'photo_807': 'https://pp.userapi.com/c834402/v834402946/14ed98/z7mHcsrn1jI.jpg',
      'post_id': 386,
      'text': '',
      'user_id': 100,
      'width': 1600},
     'type': 'photo'},
    {'link': {'description': 'Национальный исследовательский университет «Высшая школа экономики» (

Помимо текста поста можно найти много всего интересного. Например, тип поста (`post_type`), дата (`date`), id поста (`id`), лайки (`likes`, которые включают информацию о том, могут ли пользователи лайкать пост и публиковать его, а также собственно число лайков), репосты (`reposts`, которые включают число репостов), число просмотров (`views`), комментарии (`comments`, которые включают информацию о том, могут ли пользователи комментировать пост, и число комментариев), и так далее.

Давайте остановимся на тексте поста, id автора, id поста и дате публикации. Чтобы извлечь соответствующую информацию, сохраним `items` и извлечем из них нужные поля:

In [15]:
items = res['items']

In [16]:
full_list = []

for item in items:
    l = [item['from_id'], item['id'], item['text'], item['date']] # нужные поля
    full_list.append(l) # добавляем в список списков full_list

In [18]:
# несколько элементов списка
full_list[0:3]

[[-132, 32053, '', 1527716601],
 [-132, 32052, '', 1527716594],
 [-132,
  32051,
  'Компания КРОК приглашает:\nПривет! Этим летом тебя ждёт IT-жара! \nУспей подать заявку в «Летние школы» КРОК, чтобы не пропустить полезный нетворкинг, получить новые знания и опыт работы с одной из крупнейших IT-компаний в России. \nНаправления подготовки: \n• разработка программного обеспечения с 9 по 20 июля \n• телекоммуникации с 2 по 13 июля \n• построение IT-инфраструктуры современных компаний с 16 по 27 июля \n• разработка многомерного хранилища данных с 15 по 29 июля, с 6 по 20 августа \nКто может принять участие? Студенты 2-6 курсов технических вузов. \nСрок обучения: 2-3 недели (в июле – августе). \nГде? В офисе КРОК в Москве (м. Площадь Ильича/ Римская, ул. Волочаевская, д. 5, корп. 1) \nПрием заявок: до 20 июня 2018 г. \nВажно! Участие бесплатное. \nУзнать более детальную информацию о программе и зарегистрироваться ты можешь по ссылке. http://www.croc.ru/vacancy/students/detail/79855/\nДо вст

Видно, что в двух первых постах текста не обнаружено, там только картинки и перепосты. 

Из этого списка списков можно легко сделать датафрейм `pandas`. Но прежде посмотрим, как сгрузить следующие 100 (и не только 100) постов со стены. Обычно при работе с API нужно принимать во внимание две вещи: 1) какое ограничение стоит на число запросов за один раз (число постов в нашем случае); 2) какое ограничение стоит на число запросов в минуту. Чтобы действовать в соответствии с этими ограничениями, поступим так: будем грузить каждые следующие 100 постов, добавлять их к нашему списку, потом немного ждать, и грузить еще 100 постов, и так до тех пор, пока не сгрузим желаемое количество.

Давайте для начала выберем это желаемое число постов. Пусть будет 400. 

In [21]:
nposts = 400

Теперь вопрос: по каким значениям нужно "пробегаться" в цикле, чтобы сгрузить посты с 100 по 400 (первые 100 уже сохранены в `res`)? По целым значениям от 2 до 4 включительно, умножая эти значения на 100. В `vk.get` есть опция `offset`. Она позволяет "сдвинуть" начало отсчета постов на некоторое число. Так, если мы напишем `api.wall.get(domain = group, count = 100, offset = 100, v = 5.73)`,  мы получим посты с 100 по 200, так как начало отсчета сдвинулось на 100.

Реализуем описанное выше. Для цикла нам понадобится `range()`, а для задержки после выгрузки каждой сотни постов нам пригодится функция `sleep`: 

In [22]:
from time import sleep

In [24]:
for i in range(2, int(nposts/100) + 1):
    res2 = api.wall.get(domain = group, count = 100, offset = 100 * i, v = 5.73)
    items2 = res2['items']
    items.extend(items2) # добавляем к первой сотне постов в items
    sleep(10)

In [31]:
# опять выберем только нужные поля
full_list = []
for item in items:
    l = [item['from_id'], item['id'], item['text'], item['date']]
    full_list.append(l)

Оставлось превратить обновленный список `items` (список списков) в датафрейм. Импортируем `pandas`.

In [25]:
import pandas as pd

Создадим датафрейм:

In [34]:
df = pd.DataFrame(full_list)

In [36]:
df.head(10)

,0,1,2,3
0,-132,32053,,1527716601
1,-132,32052,,1527716594
2,-132,32051,Компания КРОК приглашает:\nПривет! Этим летом ...,1527538730
3,-132,32050,Сбываются мечты Е.Г.Ясина. Вышка выращивает не...,1527080728
4,-132,32049,,1527078739
5,-132,32048,,1527078368
6,-132,32047,"Павел Алферов, Дмитрий Маев, Valery Anshin, Ни...",1527078310
7,-132,32046,,1526849884
8,-132,32045,,1525030127
9,-132,32044,Вакансия для студентов Вышки от коллег из ADID...,1524946182


Ура! Осталось только дать внятные названия столбцам и разобраться, почему дата представлена в таком виде. что делать со столбцами, мы уже знаем.

In [37]:
df.columns = ['from_id', 'id', 'text', 'date']
df.head(10)

,from_id,id,text,date
0,-132,32053,,1527716601
1,-132,32052,,1527716594
2,-132,32051,Компания КРОК приглашает:\nПривет! Этим летом ...,1527538730
3,-132,32050,Сбываются мечты Е.Г.Ясина. Вышка выращивает не...,1527080728
4,-132,32049,,1527078739
5,-132,32048,,1527078368
6,-132,32047,"Павел Алферов, Дмитрий Маев, Valery Anshin, Ни...",1527078310
7,-132,32046,,1526849884
8,-132,32045,,1525030127
9,-132,32044,Вакансия для студентов Вышки от коллег из ADID...,1524946182


С датой все интереснее. То, что указано в столбце `date`, это дата в виде UNIX-времени (POSIX-времени). Это число секунд, прошедших с 1 января 1970 года. Несмотря на то, что такой формат даты-времени кажется необычным, он довольно широко распространен в разных системах и приложениях (см. подробнее [здесь](https://ru.wikipedia.org/wiki/Unix-%D0%B2%D1%80%D0%B5%D0%BC%D1%8F)). Этот факт, конечно, радует, но хочется получить дату в более человеческом формате. Давайте напишем функцию для перевода UNIX-времени в формат *год-месяц-день-часы-минуты-секунды*. Для этого нам понадобится модуль *datetime*.

In [42]:
from datetime import datetime

In [43]:
def date_norm(date):
    d = datetime.fromtimestamp(date) # timestamp - UNIX-время в виде строки
    str_d = d.strftime("%Y-%m-%d %H:%M:%S") # %Y-%m-%d %H:%M:%S - год-месяц-день, часы:минуты:секунды
    date_norm, time_norm = str_d.split(' ') # разобьем результат на части, отделим дату от времени
    return date_norm, time_norm

Применим нашу функцию к элементам столбца *date* и создадим новый ‒ *date_norm*.

In [44]:
df['date_norm'] = df.date.apply(date_norm)

In [45]:
df.head()

,from_id,id,text,date,date_norm
0,-132,32053,,1527716601,"(2018-05-31, 00:43:21)"
1,-132,32052,,1527716594,"(2018-05-31, 00:43:14)"
2,-132,32051,Компания КРОК приглашает:\nПривет! Этим летом ...,1527538730,"(2018-05-28, 23:18:50)"
3,-132,32050,Сбываются мечты Е.Г.Ясина. Вышка выращивает не...,1527080728,"(2018-05-23, 16:05:28)"
4,-132,32049,,1527078739,"(2018-05-23, 15:32:19)"


Можно было, конечно, не разбивать на части дату и время, сохранять одной строкой. А можно написать функции, которые будут отделять дату от времени ‒ извлекать их из кортежа в `date_norm`.

In [46]:
def get_date(date):
    return date[0]

def get_time(date):
    return date[1]

In [49]:
df['Date'] = df.date_norm.apply(get_date)
df['Time'] = df.date_norm.apply(get_time)

In [50]:
df.head()

,from_id,id,text,date,date_norm,Date,Time
0,-132,32053,,1527716601,"(2018-05-31, 00:43:21)",2018-05-31,00:43:21
1,-132,32052,,1527716594,"(2018-05-31, 00:43:14)",2018-05-31,00:43:14
2,-132,32051,Компания КРОК приглашает:\nПривет! Этим летом ...,1527538730,"(2018-05-28, 23:18:50)",2018-05-28,23:18:50
3,-132,32050,Сбываются мечты Е.Г.Ясина. Вышка выращивает не...,1527080728,"(2018-05-23, 16:05:28)",2018-05-23,16:05:28
4,-132,32049,,1527078739,"(2018-05-23, 15:32:19)",2018-05-23,15:32:19


Всё! Материалы о разных методах и функциях для `vk.api` можно найти в [официальной документации](https://vk.com/dev/manuals). 